# R Aqr JWST MIRI coronagraph offset
Calculations of X,Y offset for R Aqr offset target acquisition
for MIRI coronagraph.

In [1]:
# Set up autoloader
%load_ext autoreload
%autoreload 2

In [2]:
# Set up modules and stuff
import numpy as np
import astropy.units as u
import compute_offsets as coff
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

Matplotlib is building the font cache; this may take a moment.


## JWST Parameters
This is the date range for the scheduled JWST observation, and camera configuration.

In [3]:
# JWST PA angle of the telescope V3 axis at time of observation
V3PA_TIME1 = 320.074 # V3PA at the beginning of JWST observing window
V3PA_TIME2 = 320.074 # V3PA at the end of JWST observing window

# Coronographic mask ids
CORON_ID_A = '1140' # ID for first coronagraphic observation
CORON_ID_B = '1550' # ID for second coronagraphic observation

In [4]:
# JWST dates and times
JWST_TIME1 = '2024-10-30T18:19:00.000'    # Beginning of JWST observing window
JWST_TIME2 = '2024-12-12T00:19:55.000'    # Beginning of JWST observing window

# Create Time-like objects with the JWST times
jwst_time1 = Time(JWST_TIME1, format='isot', scale='utc')
jwst_time2 = Time(JWST_TIME2, format='isot', scale='utc')

## Reference coordinates for R Aqr and offset star
We provide the most recent set of coordinates and proper motion for R Aqr based on our VLBI observations. The acquisition star coordinates are from Gaia DR3.

In [7]:
# R Aqr VLBI coordinates and proper motion
RA = '23:43:49.50785 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.911 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
#PARALLAX = 2.5931              # Gaia DR3 Parallax (mas)
PARALLAX = 4.59                 # Adriantsaralaza et al. 2022 parallax (mas)
OBS_DATE = '2024-04-07'         # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr2 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE, format='iso', scale='utc'),
                  frame=FRAME
                 )

In [8]:
# Gaia DR3 2419576255768131328 coordinates
RA = '355.9498781532638 degrees'     # RA in decimal degrees (ICRS)
DEC = '-15.283947951921947 degrees'  # DEC in decimal degrees (ICRS) 
PMRA = 20.269224605443817            # Proper motion in RA (mas/yr)
PMDEC = -11.812614959581758          # Proper motion in DEC (mas/yr)
PARALLAX = 3.7063351210713535        # Gaia DR3 Parallax (mas)
OBS_DATE = 2016.0                    # Observation date
EPOCH = 2016.0                       # Reference epoch
FRAME = 'icrs'                       # Coordinate frame

# Create Skycoord object with the given coordinates
acqt = SkyCoord(RA,
                DEC,
                pm_ra_cosdec=PMRA*u.mas/u.yr,
                pm_dec=PMDEC*u.mas/u.yr,
                distance=Distance(parallax=PARALLAX*u.mas),
                equinox=Time(EPOCH, format='jyear'),
                obstime=Time(OBS_DATE, format='jyear', scale='tcb'),
                frame=FRAME
            )